In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

# Exploratory Data Analysis (EDA)
# Keşifsel Veri Analizi (EDA)

In [ ]:
train = pd.read_csv('../input/competitive-data-science-predict-future-sales/sales_train.csv', parse_dates = ['date'], infer_datetime_format = True, dayfirst = True)
test=pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/test.csv")
item_categories=pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv")
items=pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/items.csv")
sample_submission=pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv")
shops=pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/shops.csv")

In [ ]:
print(f'sales_train verilerindeki satır sayısı: {train.shape[0]}')
print(f'sales_train verilerindeki sütun sayısı: {train.shape[1]}')
print(f'sales_train verilerindeki değer sayısı: {train.count().sum()}')
print(f'sales_train verilerindeki eksik değerlerin sayısı: {sum(train.isna().sum())}')
train.head()

In [ ]:
print(f'test verilerindeki satır sayısı: {test.shape[0]}')
print(f'test verilerindeki sütun sayısı: {test.shape[1]}')
print(f'test verilerindeki değer sayısı: {test.count().sum()}')
print(f'test verilerindeki eksik değerlerin sayısı: {sum(test.isna().sum())}')
item_categories.head()

In [ ]:
print(f'items verilerindeki satır sayısı: {items.shape[0]}')
print(f'items verilerindeki sütun sayısı: {items.shape[1]}')
print(f'items verilerindeki değer sayısı: {items.count().sum()}')
print(f'items verilerindeki eksik değerlerin sayısı: {sum(items.isna().sum())}')
items.head()

In [ ]:
print(f'sample_submission verilerindeki satır sayısı: {sample_submission.shape[0]}')
print(f'sample_submission verilerindeki sütun sayısı: {sample_submission.shape[1]}')
print(f'sample_submission verilerindeki değer sayısı: {sample_submission.count().sum()}')
print(f'sample_submission verilerindeki eksik değerlerin sayısı: {sum(sample_submission.isna().sum())}')
sample_submission.head()

In [ ]:
print(f'shops verilerindeki satır sayısı: {shops.shape[0]}')
print(f'shops verilerindeki sütun sayısı: {shops.shape[1]}')
print(f'shops verilerindeki değer sayısı: {shops.count().sum()}')
print(f'shops verilerindeki eksik değerlerin sayısı: {sum(shops.isna().sum())}')
shops.head()

In [ ]:
print(f'item_categories verilerindeki satır sayısı: {item_categories.shape[0]}')
print(f'item_categories verilerindeki sütun sayısı: {item_categories.shape[1]}')
print(f'item_categories verilerindeki değer sayısı: {item_categories.count().sum()}')
print(f'item_categories verilerindeki eksik değerlerin sayısı: {sum(item_categories.isna().sum())}')
item_categories.head()

#### Bring all the data together to create a clear EDA and find some compelling insights from the data.
#### Net bir EDA oluşturmak ve verilerden bazı ilgi çekici içgörüler bulmak için tüm verileri bir araya getirin.


In [ ]:
df_eda = pd.merge(train, shops, how='left', left_on = 'shop_id', right_on = 'shop_id')
df_eda = pd.merge(df_eda, items, how='left', left_on='item_id', right_on='item_id')
df_eda = pd.merge(df_eda, item_categories, how='left', left_on='item_category_id', right_on='item_category_id')
df_eda.drop(['item_id', 'item_category_id', 'shop_id'], axis=1, inplace=True)
df_eda.head()

In [ ]:
print(df_eda.shape)

In [ ]:
df_eda.columns

In [ ]:
df_eda.info()

#### change the data type of column 'date'
#### 'tarih' sütununun veri türünü değiştirin

In [ ]:
df_eda['date'] = pd.to_datetime(df_eda['date'])
df_eda.info()

#### Check if we have duplicate rows.
#### Yinelenen satırlarımız olup olmadığını kontrol edin.

In [ ]:
df_eda.duplicated().sum()

In [ ]:
# Yinelenen satırlara bir göz atın (ilk satırlar)
df_eda[df_eda.duplicated(keep='first')]

#### Take a look at the duplicate lines (second lines)
#### Yinelenen satırlara bir göz atın (ikinci satırlar)

In [ ]:
df_eda[df_eda.duplicated(keep='last')]

In [ ]:
# Yinelenen satırları silme
df_eda.drop_duplicates(keep='first', inplace=True)
df_eda.duplicated().sum()

# Data Viz
# Veri Görselleştirme

#### Mağaza adına göre satışlara göz atmak için bar grafiği oluşturma


In [ ]:
plt.figure(figsize=(10, 25))
sns.countplot(y=df_eda['shop_name'])
plt.ylabel('Mağaza adı')
plt.title('Farklı mağazalarda satılan ürün miktarı')
plt.show()

#### Find and plan your top 10 products
#### En çok satan 10 ürünü bulun ve planlayın

In [ ]:
top_10_selling_item = df_eda['shop_name'].value_counts().head(10)
top_10_selling_item = top_10_selling_item.to_frame().reset_index()
top_10_selling_item.columns = ['shop_name', 'count']
plt.figure(figsize=(13, 5))
sns.barplot(data=top_10_selling_item, x='count', y='shop_name')
plt.title('En çok satan 10 ürün.')
plt.ylabel('Mağaza Adı')
plt.xlabel('Sayı')
plt.show()

#### Find and plot top 10 selling categories of products
#### En çok satan 10 ürün kategorisini bulun ve planlayın

In [ ]:
top_10_selling_category = df_eda['item_category_name'].value_counts().head(10)
top_10_selling_category = top_10_selling_category.to_frame().reset_index()
top_10_selling_category.columns = ['item_name', 'count']
plt.figure(figsize=(13, 5))
sns.barplot(data=top_10_selling_category, x='count', y='item_name')
plt.title('En çok satan 10 kategori.')
plt.ylabel('Öğe adı')
plt.xlabel('Sayı')
plt.show()

#### Function to plot trends from column list.
#### Sütun listesinden eğilimleri çizme işlevi.

In [ ]:
def plot_trend(columns):
    for column in columns:
        movies = df_eda[df_eda['item_category_name'] == column]
        movies = movies.groupby('date_block_num')['item_cnt_day'].sum().to_frame().reset_index()
        plt.figure(figsize=(20,4))
        plt.axhline (y =movies['item_cnt_day'].mean(), color='red', linewidth = 1, linestyle ='dashed', label = 'Tüm zamanlar için ortalama satışlar.')
        sns.lineplot(data=movies, x='date_block_num', y='item_cnt_day', label = 'Trend line.')
        plt.title('Satın alma eğilimi ' + str(column) + '.', fontsize=14)
        plt.legend(loc = 'upper right')
        plt.xlabel('Ay Sayısı (Ocak 2013ten Ekim 2015e kadar)')
        plt.ylabel('Satılan ürünler')
        plt.show()
        print('\n')

#### Calling the function from the previous cell
#### Fonksiyonu önceki hücreden çağırma

In [ ]:
plot_trend(list(top_10_selling_category['item_name']))

### Here we can clearly see the trend increasing in the category "gifts", "board games" and "games for ps4", while in other categories the trend seems to decrease

### Burada "hediyeler", "masa oyunları" ve "ps4 için oyunlar" kategorisinde trendin arttığını açıkça görebiliriz, diğer kategorilerde trendin düştüğü görülüyor

#### looking at the number of different categories
#### farklı kategori sayısına bakmak

In [ ]:
plt.rcParams['figure.figsize'] = (22, 12)
sns.barplot(items['item_category_id'], items['item_id'], palette = 'colorblind')
plt.title('Farklı Ürün Kategorileri için Sayım', fontsize = 30)
plt.xlabel('Ürün Kategorileri', fontsize = 15)
plt.ylabel('Her Kategorideki Öğeler', fontsize = 15)
plt.show()

#### take a look at the daily distribution of products sold
#### günlük satılan ürün dağılımına bir göz atmak

In [ ]:
plt.rcParams['figure.figsize'] = (20, 10)
sns.countplot(train['date_block_num'])
plt.title('Aylara göre tarih blokları', fontsize = 30)
plt.xlabel('Farklı ay blokları', fontsize = 15)
plt.ylabel('Numara. Satın Alma Sayısı', fontsize = 15)
plt.show()

#### converting data to monthly sales data
#### verileri aylık satış verilerine dönüştürme

In [ ]:
# sadece aylık satış verileriyle bir veri seti oluşturma
# create a dataset with only monthly sales data
data = train.groupby([train['date'].apply(lambda x: x.strftime('%Y-%m')),'item_id','shop_id']).sum().reset_index()

In [ ]:
# verilere eklemek istediğimiz önemli özellikleri belirtmek
# specify the important features we want to add to the data
data = data[['date','item_id','shop_id','item_cnt_day']]

In [ ]:
# sonunda veri kümesinden önemli olan belirli özellikleri seçebiliriz
# finally we can select specific features from the dataset that are important
data = data.pivot_table(index=['item_id','shop_id'], columns = 'date', values = 'item_cnt_day', fill_value = 0).reset_index()

In [ ]:
# yeni hazırlanan veri setine bakılıyor
# looking at newly prepared dataset
data.shape

In [ ]:
# Hazırlanan aylık satış verilerini test veri seti ile birleştirelim
# Let's combine the prepared monthly sales data with the test data set
test = pd.merge(test, data, on = ['item_id', 'shop_id'], how = 'left')

In [ ]:
# veri setinde bulunan boş değerlerin doldurulması
# filling in blanks in the dataset
test.fillna(0, inplace = True)

In [ ]:
# veri setini kontrol etme
# check the dataset
test.head()

# Model Building
# Model Oluşturma

#### now let's generate real training data
#### şimdi gerçek eğitim verilerini oluşturalım

In [ ]:
x_train = test.drop(['2015-10', 'item_id', 'shop_id'], axis = 1)
y_train = test['2015-10']

#### delete the first column so it can predict future sales data
#### gelecekteki satış verilerini tahmin edebilmesi için ilk sütunu silme

In [ ]:
x_test = test.drop(['2013-01', 'item_id', 'shop_id'], axis = 1)

In [ ]:
# veri setlerinin şekillerini kontrol etme
# checking the shapes of datasets
print("Shape of x_train :", x_train.shape)
print("Shape of x_test :", x_test.shape)
print("Shape of y_test :", y_train.shape)

In [ ]:
# x_train veri setini kontrol edelim
# Let's check the x_train dataset
x_train.head()

In [ ]:
# x_test verilerini kontrol edelim
# Let's check the x_test data
x_test.head()

#### split data into train and current dataset
#### verileri tren ve geçerli veri kümesine ayırma

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size = 0.2, random_state = 0)

In [ ]:
print("Shape of x_train :", x_train.shape)
print("Shape of x_valid :", x_valid.shape)
print("Shape of y_train :", y_train.shape)
print("Shape of y_valid :", y_valid.shape)

## LGBMRegressor

In [ ]:
from lightgbm import LGBMRegressor

model_lgb = LGBMRegressor( n_estimators=200,
                           learning_rate=0.03,
                           num_leaves=32,
                           colsample_bytree=0.9497036,
                           subsample=0.8715623,
                           max_depth=8,
                           reg_alpha=0.04,
                           reg_lambda=0.073,
                           min_split_gain=0.0222415,
                           min_child_weight=40)
model_lgb.fit(x_train, y_train)

y_pred_lgb = model_lgb.predict(x_test)

In [ ]:
y_pred_lgb = model_lgb.predict(x_test).clip(0., 20.)

#### Create and send the submission file
#### Gönderim dosyasını oluşturun ve gönderin

In [ ]:
preds = pd.DataFrame(y_pred_lgb, columns=['item_cnt_month'])
preds.to_csv('submission.csv',index_label='ID')

# 2.Model Building And Data Preprocessing (Second Best Model Score)
# 2.Model Oluşturma Ve Veri Ön İşleme (İkinci En İyi Model Puanı)

In [ ]:
import numpy as np
import pandas as pd
import tkinter as tk
from tkinter import *
from tkinter import ttk
from PIL import ImageTk,Image
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import string
from nltk.corpus import *
from wordcloud import *
from sklearn import *
from sklearn.model_selection import *
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import *
from sklearn.decomposition import *
from sklearn.tree import *
from sklearn.feature_extraction.text import *
from sklearn.linear_model import *
from sklearn.ensemble import *
import xgboost as xgb
from sklearn.metrics import *
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [ ]:
item_categories,items,sales_train,sample,shops,test=pd.read_csv('../input/competitive-data-science-predict-future-sales/item_categories.csv'),\
pd.read_csv('../input/competitive-data-science-predict-future-sales/items.csv'), \
    pd.read_csv('../input/competitive-data-science-predict-future-sales/sales_train.csv'),\
pd.read_csv('../input/competitive-data-science-predict-future-sales/sample_submission.csv'),\
        pd.read_csv('../input/competitive-data-science-predict-future-sales/shops.csv'),\
pd.read_csv('../input/competitive-data-science-predict-future-sales/test.csv')

In [ ]:
item_categories.head()

In [ ]:
items.head()

In [ ]:
items_df=pd.merge(items,item_categories,how='left',left_on='item_category_id',right_on='item_category_id')
items_df.head()

In [ ]:
sales_train.head()

In [ ]:
train_df=pd.merge(sales_train,items_df,how='left',left_on='item_id',right_on='item_id')
train_df.head()

In [ ]:
shops.head()

In [ ]:
train_df=pd.merge(train_df,shops,how='left',left_on='shop_id',right_on='shop_id')
train_df.head()

In [ ]:
train_df['year'] = pd.DatetimeIndex(train_df['date']).year
train_df['month'] = pd.DatetimeIndex(train_df['date']).month
train_df['day'] = pd.DatetimeIndex(train_df['date']).day

In [ ]:
train_df.drop('date',axis=1,inplace=True)

In [ ]:
fig, ax = plt.subplots(figsize=(15,7))
plt.xticks([i for i in range(1,13)])
train_df.groupby(train_df[train_df['year']==2013].month).count()['item_id'].plot(ax=ax,label=2013)
train_df.groupby(train_df[train_df['year']==2014].month).count()['item_id'].plot(label=2014)
train_df.groupby(train_df[train_df['year']==2015].month).count()['item_id'].plot(label=2015)
plt.grid("on")
plt.title("Öğe Satış Aylık")
plt.legend()

In [ ]:
sns.jointplot(x="item_cnt_day", y="item_price", data=train_df)
plt.show()

In [ ]:
train_df.isnull().sum()

In [ ]:
train_df.info()

In [ ]:
train_df.shape

In [ ]:
train_df.drop_duplicates()

In [ ]:
train_df.columns

In [ ]:
train_df.drop(train_df[train_df.item_price > 3 * train_df.item_price.std()].index, inplace = True)
train_df.drop(train_df[train_df.item_cnt_day > 3 * train_df.item_cnt_day.std()].index, inplace = True)
train_df.drop(train_df[train_df.item_price < 0].index, inplace = True)
train_df.drop(train_df[train_df.item_cnt_day < 0].index, inplace = True)

In [ ]:
train_df['revenue'] = train_df.item_price * train_df.item_cnt_day

In [ ]:
df=train_df.pivot_table(index=['shop_id', 'item_id'], columns='date_block_num', values='item_cnt_day', aggfunc='sum').fillna(0.0)
df.head()

In [ ]:
train_df = df.reset_index()
train_df["shop_id"] = train_df.shop_id.astype("str")
train_df["item_id"] = train_df.item_id.astype("str")
train_df.head()

In [ ]:
item_cat_df = items.merge(item_categories[['item_category_id','item_category_name']], how="inner", on="item_category_id")[['item_id','item_category_name']]

In [ ]:
item_cat_df['item_id'] = item_cat_df.item_id.astype('str')

In [ ]:
train_df = train_df.merge(item_cat_df, how="inner", on="item_id")

In [ ]:
train_df.dropna(axis=1,inplace=True)

In [ ]:
EN = preprocessing.LabelEncoder()
train_df['item_category_name'] = EN.fit_transform(train_df['item_category_name'])
train_df.head()

In [ ]:
train_df['item_category_name'] = EN.fit_transform(train_df['item_category_name'])
train_df = train_df[['shop_id', 'item_id', 'item_category_name'] + list(range(34))]
train_df.head()

In [ ]:
param = {'max_depth': 15, 
         'subsample': 1,
         'min_child_weight': 1,
         'eta':0.34, 
         'seed':1,
         'verbosity': 1,
         'eval_metric':'rmse'}



xgbtrain = xgb.DMatrix(train_df.iloc[:,  (train_df.columns != 33)].values,
                       train_df.iloc[:, train_df.columns == 33].values)

model = xgb.train(param, xgbtrain)
preds = model.predict(xgb.DMatrix(train_df.iloc[:,  (train_df.columns != 33)].values))
RMSE = np.sqrt(mean_squared_error(preds,train_df.iloc[:, train_df.columns == 33].values))

print(RMSE)

In [ ]:
fig, ax = plt.subplots(figsize = (20, 20))
xgb.plot_importance(model, ax = ax)

In [ ]:
test.shop_id = test.shop_id.astype('str')
test.item_id = test.item_id.astype('str')

In [ ]:
test = test.merge(train_df, how="left", on=["shop_id", "item_id"]).fillna(0.0)
test.head()

In [ ]:
d = dict(zip(test.columns[4:],list(np.array(list(test.columns[4:])) - 1)))

test = test.rename(d, axis = 1)

In [ ]:
predictions = model.predict(xgb.DMatrix(test.iloc[:, (test.columns != 'ID') & (test.columns != -1)].values))

In [ ]:
predictions = list(map(lambda x: min(20, max(x,0)), list(predictions)))
submission = pd.DataFrame({'ID': test.ID, 'item_cnt_month': predictions})
submission.to_csv('submission2.csv', index=False)

# 3. Model Building And Data Preprocessing (The Best Score)
# 3. Model Oluşturma ve Veri Ön İşleme (En İyi Skor Olan Model)

In [ ]:
import numpy as np
import pandas as pd
import warnings
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
warnings.filterwarnings(action='ignore') # Ignore warning message

In [ ]:
# date path
data_path = '/kaggle/input/competitive-data-science-predict-future-sales/'
sales_train = pd.read_csv(data_path + 'sales_train.csv')
shops = pd.read_csv(data_path + 'shops.csv')
items = pd.read_csv(data_path + 'items.csv')
item_categories = pd.read_csv(data_path + 'item_categories.csv')
test = pd.read_csv(data_path + 'test.csv')
submission = pd.read_csv(data_path + 'sample_submission.csv')

In [ ]:
def downcast(df, verbose=True):
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        dtype_name = df[col].dtype.name
        if dtype_name == 'object':
            pass
        elif dtype_name == 'bool':
            df[col] = df[col].astype('int8')
        elif dtype_name.startswith('int') or (df[col].round() == df[col]).all():
            df[col] = pd.to_numeric(df[col], downcast='integer')
        else:
            df[col] = pd.to_numeric(df[col], downcast='float')
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose:
        print('{:.1f}% compressed'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

all_df = [sales_train, shops, items, item_categories, test]
for df in all_df:
    df = downcast(df)

In [ ]:
daily = sales_train[['date', 'date_block_num', 'shop_id', 'item_id', 'item_cnt_day']]
daily = daily.groupby(['date'], as_index=False)
daily = daily.agg({'item_cnt_day':['sum']})
daily.head()

In [ ]:
monthly_per_shop = sales_train[['date', 'date_block_num', 'shop_id', 'item_id', 'item_cnt_day']]
monthly_per_shop = monthly_per_shop.groupby(['date_block_num','shop_id'], as_index=False)
monthly_per_shop = monthly_per_shop.agg({'item_cnt_day':['sum']})
monthly_per_shop['item_cnt_month']  = monthly_per_shop['item_cnt_day']['sum']
monthly_per_shop = monthly_per_shop.drop(['item_cnt_day'], axis=1)
monthly_per_shop.head()

In [ ]:
fig, axes = plt.subplots(2, 1)
plt.subplots_adjust(hspace=0.5)

flierprops = dict(marker='o', markerfacecolor='cornflowerblue', markersize=6, markeredgecolor='navy')

_ = axes[0].boxplot(x=sales_train.item_cnt_day, flierprops=flierprops, vert=False)
_ = axes[1].boxplot(x=sales_train.item_price, flierprops=flierprops, vert=False)

_ = axes[0].set_title('item_cnt_day')
_ = axes[1].set_title('item_price')

In [ ]:
# Item_price değeri 0'dan büyük olan verileri ayıklayın
sales_train = sales_train[sales_train['item_price'] > 0]

In [ ]:
# Item_priceof 50.000'den az olan verileri ayıklayın
sales_train = sales_train[sales_train['item_price'] < 50000]

In [ ]:
# item_cnt_day 0'dan büyük olan verileri çıkar
sales_train = sales_train[sales_train['item_cnt_day'] > 0]

In [ ]:
# item_cnt_day değeri 1.000'den az olan verileri ayıklayın
sales_train = sales_train[sales_train['item_cnt_day'] < 1000]

In [ ]:
# Bir dükkanın adını düzeltin
shops.loc[shops.shop_name == 'Сергиев Посад ТЦ "7Я"',"shop_name"] = 'СергиевПосад ТЦ "7Я"'

In [ ]:
# Shop_name'nin ilk kısmı şehirdir, örn.Serguiev Possad
shops["city"] = shops.shop_name.str.split(' ').map(lambda x: x[0])

In [ ]:
# Shop_name'nin ikinci kısmı kategoridir, örn. alışveriş Merkezi
shops["shop_category"] = shops.shop_name.str.split(" ").map(lambda x: x[1])
shops.loc[shops.city == "!Якутск", "city"] = "Якутск" 

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Create Label Encoder
label_encoder = LabelEncoder()
# City Feature Label Encoding 
shops['city'] = label_encoder.fit_transform(shops['city'])
# Shop Category Label Encoding 
shops['shop_category'] = label_encoder.fit_transform(shops['shop_category'])

In [ ]:
# Remove shop_name feature
shops = shops.drop('shop_name', axis=1)

shops.head()

In [ ]:
#  item_name özelliğini kaldır
items = items.drop(['item_name'], axis=1)

In [ ]:
# Özellik olarak ürünün ilk satıldığı tarihi oluşturun
items['first_sale_date'] = sales_train.groupby('item_id').agg({'date_block_num': 'min'})['date_block_num']

items.head()

In [ ]:
items[items['first_sale_date'].isna()]

In [ ]:
# Replace NaN of first_sale_date with 34
items['first_sale_date'] = items['first_sale_date'].fillna(34)

In [ ]:
# item_categories_name öğesinin ilk kelimesini kategoriye çıkarın
item_categories['category'] = item_categories['item_category_name'].apply(lambda x: x.split()[0])  
item_categories['category'].value_counts()

In [ ]:
def make_etc(x):
    if len(item_categories[item_categories['category']==x]) >= 5:
        return x
    else:
        return 'etc'

In [ ]:
# Replace with 'etc' if category count is less than 5
item_categories['category'] = item_categories['category'].apply(make_etc)

In [ ]:
# Create Label Encoder
label_encoder = LabelEncoder()

In [ ]:
# Kategori Özellik Etiketi Kodlaması
item_categories['category'] = label_encoder.fit_transform(item_categories['category'])

In [ ]:
# item_category_name özelliğini kaldır
item_categories = item_categories.drop('item_category_name', axis=1)

In [ ]:
from itertools import product

train = []
# Create date_block_num, shop_id, item_id combination
for i in sales_train['date_block_num'].unique():
    all_shop = sales_train.loc[sales_train['date_block_num']==i, 'shop_id'].unique()
    all_item = sales_train.loc[sales_train['date_block_num']==i, 'item_id'].unique()
    train.append(np.array(list(product([i], all_shop, all_item))))

In [ ]:

idx_features = ['date_block_num', 'shop_id', 'item_id'] # base features
train = pd.DataFrame(np.vstack(train), columns=idx_features)

In [ ]:
group = sales_train.groupby(idx_features).agg({'item_cnt_day': 'sum',
                                               'item_price': 'mean'})
group = group.reset_index()
group = group.rename(columns={'item_cnt_day': 'item_cnt_month', 'item_price': 'item_price_mean'})


In [ ]:

train = train.merge(group, on=idx_features, how='left')

train.head()

In [ ]:
import gc

# grup değişkeni çöp toplama
del group
gc.collect();
# Satılan ürün sayısı için bir özellik ekleyin
group = sales_train.groupby(idx_features).agg({'item_cnt_day': 'sum'})
group = group.reset_index()
group = group.rename(columns={'item_cnt_day': 'item_count'})

train = train.merge(group, on=idx_features, how='left')

# Garbage collection
del group, sales_train
gc.collect()

train.head()

In [ ]:
# Test verisi tarih_blok_sayısı'nı 34 olarak ayarla
test['date_block_num'] = 34

# Concatenate train and test
all_data = pd.concat([train, test.drop('ID', axis=1)],
                     ignore_index=True,
                     keys=idx_features)
# Replace NaN with 0
all_data = all_data.fillna(0)

all_data.head()

In [ ]:
# Diğer verileri birleştir
all_data = all_data.merge(shops, on='shop_id', how='left')
all_data = all_data.merge(items, on='item_id', how='left')
all_data = all_data.merge(item_categories, on='item_category_id', how='left')

In [ ]:
# Veri düşürme
all_data = downcast(all_data)

In [ ]:
# Çöp toplama
del shops, items, item_categories
gc.collect();

In [ ]:
def resumetable(df):
    print(f'Data Shape: {df.shape}')
    summary = pd.DataFrame(df.dtypes, columns=['Dtypes'])
    summary['Null'] = df.isnull().sum().values
    summary['Uniques'] = df.nunique().values
    summary['First_values'] = df.loc[0].values
    summary['Second_values'] = df.loc[1].values
    summary['Third_values'] = df.loc[2].values
    
    return summary
resumetable(all_data)

In [ ]:
def add_mean_features(df, mean_features, idx_features):
    # Temel özellikleri kontrol edin
    assert (idx_features[0] == 'date_block_num') and \
           len(idx_features) in [2, 3]
    
    # Türetilmiş özellik adını ayarla 
    if len(idx_features) == 2:
        feature_name = idx_features[1] + '_mean_sales'
    else:
        feature_name = idx_features[1] + '_' + idx_features[2] + '_mean_sales'
    
    # Temel özelliklere göre gruplandırarak aylık ortalama satış elde edin
    group = df.groupby(idx_features).agg({'item_cnt_month': 'mean'})
    group = group.reset_index()
    group = group.rename(columns={'item_cnt_month': feature_name})
    
    # df'yi idx_features'a dayalı grupla birleştir
    df = df.merge(group, on=idx_features, how='left')
    # Tarih düşürme
    df = downcast(df, False)
    # Yeni oluşturulan ortalama_özellik_adı özelliklerini ortalama_features listesine ekle
    mean_features.append(feature_name)
    
    # Çöp toplama
    del group
    gc.collect()
    
    return df, mean_features

In [ ]:
# Temel özelliklerin gruplandırılmasında 'item_id' içeren türetilmiş özelliklerin listesi
item_mean_features = []

In [ ]:
# ['date_block_num', 'item_id'] ile gruplanmış aylık ortalama satış türetilmiş özellikler oluşturun
all_data, item_mean_features = add_mean_features(df=all_data,
                                                 mean_features=item_mean_features,
                                                 idx_features=['date_block_num', 'item_id'])

In [ ]:
# ['date_block_num', 'item_id', 'city'] ile gruplanmış aylık ortalama satış türetilmiş özellikler oluşturun
all_data, item_mean_features = add_mean_features(df=all_data,
                                                 mean_features=item_mean_features,
                                                 idx_features=['date_block_num', 'item_id', 'city'])

In [ ]:
item_mean_features

In [ ]:
# Temel özelliklerin gruplandırılmasında 'shop_id' içeren türetilmiş özelliklerin listesi
shop_mean_features = []

In [ ]:
# ['date_block_num', 'shop_id', 'item_category_id'] ile gruplandırılmış aylık ortalama satış türetilmiş özellikler oluşturun
all_data, shop_mean_features = add_mean_features(df=all_data, 
                                                 mean_features=shop_mean_features,
                                                 idx_features=['date_block_num', 'shop_id', 'item_category_id'])

In [ ]:
def add_lag_features(df, lag_features_to_clip, idx_features, 
                     lag_feature, nlags=3, clip=False):
    # DataFrame'in yalnızca gecikme özelliklerini oluşturmak için gereken kısmını kopyalayın
    df_temp = df[idx_features + [lag_feature]].copy() 

    # Gecikme özellikleri oluştur
    for i in range(1, nlags+1):
        # Gecikme özelliği adı
        lag_feature_name = lag_feature +'_lag' + str(i)
        # df_temp sütun adını ayarla
        df_temp.columns = idx_features + [lag_feature_name]
        # df_temp'deki date_block_num özelliğine 1 ekleyin
        df_temp['date_block_num'] += 1
        # df'yi idx_feature'a dayalı olarak df_temp ile birleştir
        df = df.merge(df_temp.drop_duplicates(), 
                      on=idx_features, 
                      how='left')
        # Replace NaN with 0
        df[lag_feature_name] = df[lag_feature_name].fillna(0)
        # Add lag features to lag_features_to_clip to clip between 0 and 20
        if clip: 
            lag_features_to_clip.append(lag_feature_name)
    
    # Tarih düşürme
    df = downcast(df, False)
    # Çöp toplama
    del df_temp
    gc.collect()
    
    return df, lag_features_to_clip

In [ ]:
lag_features_to_clip = [] # 0 ile 20 arasında kırpılacak gecikme özelliklerinin listesi 
idx_features = ['date_block_num', 'shop_id', 'item_id'] # temel özellikler

In [ ]:
# idx_features'a dayalı olarak item_cnt_month'un 3 aylık gecikme özelliklerini oluşturun
all_data, lag_features_to_clip = add_lag_features(df=all_data, 
                                                  lag_features_to_clip=lag_features_to_clip,
                                                  idx_features=idx_features,
                                                  lag_feature='item_cnt_month', 
                                                  nlags=3,
                                                  clip=True)

In [ ]:
# idx_features'a dayalı item_count özelliğinin 3 aylık gecikme özelliklerini oluşturun
all_data, lag_features_to_clip = add_lag_features(df=all_data, 
                                                  lag_features_to_clip=lag_features_to_clip,
                                                  idx_features=idx_features,
                                                  lag_feature='item_count', 
                                                  nlags=3)



In [ ]:
# idx_features'a dayalı item_price_mean özelliğinin 3 aylık gecikme özelliklerini oluşturun
all_data, lag_features_to_clip = add_lag_features(df=all_data, 
                                                  lag_features_to_clip=lag_features_to_clip,
                                                  idx_features=idx_features,
                                                  lag_feature='item_price_mean', 
                                                  nlags=3)

In [ ]:
X_test_temp = all_data[all_data['date_block_num'] == 34]
X_test_temp[item_mean_features].sum()

In [ ]:
# dx_features tabanlı item_mean_features öğesi ile gecikme özellikleri oluşturun
for item_mean_feature in item_mean_features:
    all_data, lag_features_to_clip = add_lag_features(df=all_data, 
                                                      lag_features_to_clip=lag_features_to_clip, 
                                                      idx_features=idx_features, 
                                                      lag_feature=item_mean_feature, 
                                                      nlags=3)


In [ ]:
# item_mean_features içindeki özellikleri kaldır
all_data = all_data.drop(item_mean_features, axis=1)

In [ ]:
# ['date_block_num', 'shop_id', 'item_category_id'] temelinde shop_mean_features öğesine göre gecikme özellikleri oluşturun
for shop_mean_feature in shop_mean_features:
    all_data, lag_features_to_clip = add_lag_features(df=all_data,
                                                      lag_features_to_clip=lag_features_to_clip, 
                                                      idx_features=['date_block_num', 'shop_id', 'item_category_id'], 
                                                      lag_feature=shop_mean_feature, 
                                                      nlags=3)


In [ ]:
# shop_mean_features içindeki özellikleri kaldırın
all_data = all_data.drop(shop_mean_features, axis=1)

In [ ]:
# Remove data less than date ID 3
all_data = all_data.drop(all_data[all_data['date_block_num'] < 3].index)
all_data.head()

In [ ]:
all_data['item_cnt_month_lag_mean'] = all_data[['item_cnt_month_lag1',
                                         'item_cnt_month_lag2', 
                                         'item_cnt_month_lag3']].mean(axis=1)

In [ ]:
# Clip 0~20
all_data[lag_features_to_clip + ['item_cnt_month', 'item_cnt_month_lag_mean']] = all_data[lag_features_to_clip +['item_cnt_month', 'item_cnt_month_lag_mean']].clip(0, 20)
all_data['lag_grad1'] = all_data['item_cnt_month_lag1']/all_data['item_cnt_month_lag2']
all_data['lag_grad1'] = all_data['lag_grad1'].replace([np.inf, -np.inf], 
                                                        np.nan).fillna(0)

In [ ]:
all_data['lag_grad2'] = all_data['item_cnt_month_lag2']/all_data['item_cnt_month_lag3']
all_data['lag_grad2'] = all_data['lag_grad2'].replace([np.inf, -np.inf], 
                                                        np.nan).fillna(0)
all_data['brand_new'] = all_data['first_sale_date'] == all_data['date_block_num']
all_data['duration_after_first_sale'] = all_data['date_block_num'] - all_data['first_sale_date']
all_data = all_data.drop('first_sale_date', axis=1)
all_data['month'] = all_data['date_block_num']%12

In [ ]:
# item_price_mean, item_count özelliklerini kaldır
all_data = all_data.drop(['item_price_mean', 'item_count'], axis=1)
all_data = downcast(all_data, False) # Data downcasting
all_data.info()

In [ ]:
# Train data (Features)
X_train = all_data[all_data['date_block_num'] < 33]
X_train = X_train.drop(['item_cnt_month'], axis=1)

In [ ]:
# Valid data (Features)
X_valid = all_data[all_data['date_block_num'] == 33]
X_valid = X_valid.drop(['item_cnt_month'], axis=1)

In [ ]:
# Test data (Features)
X_test = all_data[all_data['date_block_num'] == 34]
X_test = X_test.drop(['item_cnt_month'], axis=1)

In [ ]:
# Train data (Target values)
y_train = all_data[all_data['date_block_num'] < 33]['item_cnt_month']
# Valid data (Target values)
y_valid = all_data[all_data['date_block_num'] == 33]['item_cnt_month']

In [ ]:
# Garbage collection
del all_data
gc.collect();

In [ ]:
y_train_pred = []
y_val_pred = []
y_test_pred = []
import lightgbm as lgb

# lgb hyper-parameters
params = {'metric': 'rmse',
          'num_leaves': 256,
          'learning_rate': 0.005,
          'feature_fraction': 0.75,
          'bagging_fraction': 0.75,
          'bagging_freq': 5,
          'force_col_wise' : True,
          'random_state': 10}

cat_features = ['shop_id', 'city', 'item_category_id', 'category', 'month']

# lgb treni ve geçerli veri kümesi
dtrain = lgb.Dataset(X_train, y_train)
dvalid = lgb.Dataset(X_valid, y_valid)
 
# Train LightGBM model
lgb_model = lgb.train(params=params,
                      train_set=dtrain,
                      num_boost_round=2400,
                      valid_sets=(dtrain, dvalid),
                      early_stopping_rounds=200,
                      categorical_feature=cat_features,
                      verbose_eval=100)    

In [ ]:
yhat_train_lgb = lgb_model.predict(X_train).clip(0,20)
yhat_valid_lgb = lgb_model.predict(X_valid).clip(0,20)
yhat_test_lgb = lgb_model.predict(X_test).clip(0,20)
y_train_pred.append(yhat_train_lgb)
y_val_pred.append(yhat_valid_lgb)
y_test_pred.append(yhat_test_lgb)

In [ ]:
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor, plot_importance
xgb = XGBRegressor(seed = 13, 
    tree_method = "hist", #gpu_hist
    subsample = 0.9,
    n_estimators=500,
    max_depth = 10,
    learning_rate = 0.1,
    gamma = 10,
    colsample_bytree = 0.9
    )

xgb.fit(
    X_train,y_train,
    eval_metric="rmse",
    eval_set=[(X_train, y_train), (X_valid, y_valid)],
    verbose=True,
    early_stopping_rounds = 50
    )

In [ ]:
def plot_features(booster, figsize):    
    fig, ax = plt.subplots(1,1,figsize=figsize)
    return plot_importance(booster=booster, ax=ax)
plot_features(xgb, (10,14))

In [ ]:
results = xgb.evals_result()

plt.figure(figsize=(10,7))
plt.plot(results["validation_0"]["rmse"], label="Training loss")
plt.plot(results["validation_1"]["rmse"], label="Validation loss")
plt.axvline(21, color="gray", label="Optimal tree number")
plt.xlabel("Number of trees")
plt.ylabel("Loss")
plt.legend()

In [ ]:
yhat_train_xgb = xgb.predict(X_train).clip(0, 20)
yhat_val_xgb = xgb.predict(X_valid).clip(0, 20)
print('Valdation RMSE:', mean_squared_error(y_valid, yhat_val_xgb, squared=False)) 
yhat_test_xgb = xgb.predict(X_test).clip(0, 20)

In [ ]:
# İstifleme için sonuçları ekleyin
y_train_pred.append(yhat_train_xgb)
y_val_pred.append(yhat_val_xgb)
y_test_pred.append(yhat_test_xgb)

In [ ]:
from sklearn.linear_model import LinearRegression
meta_model = LinearRegression()

In [ ]:
def stacking_preds(meta_model, train_preds, val_preds, test_preds, y_train, y_val):
    stacked_train_predictions=np.column_stack(train_preds)
    stacked_val_predictions=np.column_stack(val_preds)
    stacked_test_predictions=np.column_stack(test_preds)
    
    #Yığılmış tahminlerde #fit meta modeli
    meta_model.fit(stacked_train_predictions, y_train)
    print('RMSE =',mean_squared_error(meta_model.predict(stacked_val_predictions).clip(0, 20), y_val, squared=False))
    return meta_model.predict(stacked_test_predictions).clip(0, 20)

In [ ]:
test_preds = stacking_preds(meta_model, y_train_pred, y_val_pred, y_test_pred, y_train, y_valid)

In [ ]:
submission['item_cnt_month'] = test_preds
submission.to_csv('submission3.csv', index=False)
del X_train, y_train, X_valid, y_valid, X_test, lgb_model, dtrain, dvalid, xgb
gc.collect();